# Lab 8: Direct Lake over One Lake with Import Mode Integration

## Introduction

This advanced lab demonstrates **composite storage mode optimization** by combining Direct Lake and Import mode tables within a single Microsoft Fabric semantic model. This powerful technique allows you to leverage the best of both storage modes: Direct Lake for large-scale data access and Import mode for specific optimizations and performance tuning of critical tables.

## Lab Overview

**Learning Objectives:**
- Understand Direct Lake over One Lake vs. Direct Lake over SQL differences
- Learn to convert Direct Lake tables to Import mode within the same model
- Master hybrid storage mode configuration for optimal performance
- Analyze performance characteristics of mixed storage mode scenarios

**Key Concepts:**
- **Direct Lake over One Lake**: Advanced DirectLake implementation with OneLake integration
- **Composite Storage Modes**: Strategic combination of Direct Lake and Import modes
- **Model Cloning**: Creating optimized model variants for testing
- **Storage Mode Conversion**: Dynamic switching between Direct Lake and Import modes

**Prerequisites:** Lab 7 completion (high cardinality column optimization)

## 1. Install Semantic Link Labs Python Library
Install the Semantic Link Labs library for advanced Direct Lake and hybrid storage mode operations.

In [ ]:
# Install semantic-link-labs library for hybrid storage mode operations
%pip install -q --upgrade pip
%pip install -q semantic-link-labs azure-core==1.31.0 PyJWT==2.6.0

## 2. Install Python Libraries and Setup Parameters
Import required libraries and configure parameters for hybrid Direct Lake and Import mode operations.

In [ ]:
# Import libraries for hybrid storage mode operations and model management
import sempy_labs as labs
from sempy import fabric
import pandas as pd
import json
import time
import uuid
from sempy_labs.tom._model import TOMWrapper, connect_semantic_model

# Import specialized helper functions for advanced operations
from sempy_labs._helper_functions import (
    format_dax_object_name,
    generate_guid,
    _make_list_unique,
    resolve_dataset_name_and_id,
    resolve_workspace_name_and_id,
    _base_api,
    resolve_workspace_id,
    resolve_item_id,
    resolve_lakehouse_id,
    resolve_lakehouse_name_and_id
)

# Initialize Analysis Services for advanced model operations
fabric._client._utils._init_analysis_services()
import Microsoft.AnalysisServices.Tabular as TOM
import Microsoft.AnalysisServices

# Configure model names for hybrid storage mode testing
LakehouseName = "BigData"
SemanticModelName = f"{LakehouseName}_model"
ClonedModelName = SemanticModelName + "_clone"
workspace = None

(workspace_name, workspace_id) = resolve_workspace_name_and_id(workspace)
(lakehouse_name, lakehouse_id) = resolve_lakehouse_name_and_id(lakehouse=LakehouseName, workspace=workspace)

## 3. Clone BigData Semantic Model
Create a copy of the existing BigData semantic model for hybrid storage mode experimentation.

In [ ]:
#Clear any existing cloned model if re-running
df = fabric.list_items()
if ClonedModelName in df.values:
    model_id = df.at[df[df['Display Name'] == ClonedModelName].index[0], 'Id']
    fabric.delete_item(model_id)
    print("Cloned model deleted")

with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
    newDB = tom._tom_server.Databases.GetByName(SemanticModelName).Clone()
    newModel = tom._tom_server.Databases.GetByName(SemanticModelName).Model.Clone()
    newDB.Name = ClonedModelName
    newDB.ID = str(uuid.uuid4())
    #newDB.Model = newModel
    newModel.CopyTo(newDB.Model)
    tom._tom_server.Databases.Add(newDB)

    newDB.Update(Microsoft.AnalysisServices.UpdateOptions.ExpandFull)

## 4. Frame the Cloned Model
Refresh the cloned semantic model to ensure all data connections and relationships are properly initialized.

In [ ]:
# Refresh the cloned model to initialize data connections
labs.refresh_semantic_model(dataset=ClonedModelName)

## 5. Check Direct Lake Version
Identify whether the model uses Direct Lake over SQL (Sql.Database) or Direct Lake over One Lake (Azure.Lakehouse).

eg:  
let database = SqlDatabase(....)     = **DL/SQL**  
let database = Azure.Lakehouse(....) = **DL/OL**

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for e in tom.model.Expressions:
        print(e.Expression)

## 6. Show Storage Mode for Each Table in Cloned Model
Display the current storage mode configuration for all tables in the cloned semantic model.

In [ ]:
objects = {}
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for t in tom.model.Tables:
        #print(t.Name)
        for p in t.Partitions:
            #print(p.Mode)
            objects[t.Name] = str(p.Mode)
 
df=pd.DataFrame([objects])
display(df)

## 7. Try to Convert Direct Lake Table to Import (First Attempt)
Attempt to convert a Direct Lake table to Import mode - this will fail if using Direct Lake over SQL.

In [ ]:
# Attempt to convert Direct Lake table to Import mode (will fail if Direct Lake over SQL)
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    tom.convert_direct_lake_to_import(
        table_name="dim_Date" ,
        entity_name="dim_Date" ,
        source="BigData",
        source_type = "Lakehouse"
    )

## 8. Convert Cloned Model to Direct Lake over One Lake
Modify the cloned model to use Direct Lake over One Lake instead of Direct Lake over SQL.

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:

    #Convert import tables to Direct Lake
    for t in tom.model.Tables:
        for p in t.Partitions:
            if(p.Mode==TOM.ModeType.Import):
                t.Partitions.Remove(p)
                tom.add_entity_partition(table_name=t.Name,entity_name=t.Name)
                print(f"Table {t.Name} converted")
            p.Source.SchemaName=None

    for e in tom.model.Expressions:
        e.Expression = f"""
        let
            Source = AzureStorage.DataLake("https://onelake.dfs.fabric.microsoft.com/{workspace_id}/{lakehouse_id}", [HierarchicalNavigation=true])
        in
            Source"""
        
print("Converted semantic model to use DirectLake over One Lake")

## 9. Convert Direct Lake Table to Import (Second Attempt)
Successfully convert a Direct Lake table to Import mode now that the model uses Direct Lake over One Lake.

In [ ]:
# Convert Direct Lake table to Import mode (should work with Direct Lake over One Lake)
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    tom.convert_direct_lake_to_import(
        table_name="dim_Date" ,
        entity_name="dim_Date" ,
        source="BigData",
        source_type = "Lakehouse"
    )

## 10. Show Storage Mode for Each Table After Conversion
Display the updated storage mode configuration showing the hybrid model with both Direct Lake and Import tables.

In [ ]:
objects = {}
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for t in tom.model.Tables:
        #print(t.Name)
        for p in t.Partitions:
            #print(p.Mode)
            objects[t.Name] = str(p.Mode)
 
df=pd.DataFrame([objects])
display(df)

## Clear Schema Name
This is a temporary step to resolve issue.

Use Tabular Editor to remove/save/replace/save tables.

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for t in tom.model.Tables:
        for p in t.Partitions:
            if isinstance(p.Source,Microsoft.AnalysisServices.Tabular.EntityPartitionSource):
                p.Source.SchemaName=None

## <mark>SET CREDENTIALS AND LARGE MODEL IN SERVICE</mark>

## Refresh import table model so import table gets hydrated

In [ ]:
labs.refresh_semantic_model(dataset=ClonedModelName,tables=["dim_Date"])

## Put Relationships back after removing/adding Tables

In [ ]:
completedOK:bool=False
while not completedOK:
    try:
        with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
            #1. Remove any existing relationships
            for r in tom.model.Relationships:
                tom.model.Relationships.Remove(r)

            #2. Creates correct relationships
            tom.add_relationship(from_table="fact_myevents_1bln"                    , from_column="DateKey"     , to_table="dim_Date"       , to_column="DateKey"       , from_cardinality="Many" , to_cardinality="One")
            tom.add_relationship(from_table="fact_myevents_1bln"                    , from_column="GeographyID" , to_table="dim_Geography"  , to_column="GeographyID"   , from_cardinality="Many" , to_cardinality="One")

            tom.add_relationship(from_table="fact_myevents_2bln"                    , from_column="DateKey"     , to_table="dim_Date"       , to_column="DateKey"       , from_cardinality="Many" , to_cardinality="One")
            tom.add_relationship(from_table="fact_myevents_2bln"                    , from_column="GeographyID" , to_table="dim_Geography"  , to_column="GeographyID"   , from_cardinality="Many" , to_cardinality="One")

            tom.add_relationship(from_table="fact_myevents_1bln_partitioned_datekey", from_column="DateKey"     , to_table="dim_Date"       , to_column="DateKey"       , from_cardinality="Many" , to_cardinality="One")
            tom.add_relationship(from_table="fact_myevents_1bln_partitioned_datekey", from_column="GeographyID" , to_table="dim_Geography"  , to_column="GeographyID"   , from_cardinality="Many" , to_cardinality="One")
            completedOK=True
    except:
        print('Error adding relationships... trying again.')
        time.sleep(3)

print('done')

## Recalculate relationship indexes

In [ ]:
labs.refresh_semantic_model(dataset=ClonedModelName,refresh_type="calculate")

## Show what version of Direct Lake is being used

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for e in tom.model.Expressions:
        print(e.Expression)

## Update relationship to Many to Many (Temporarily needed)

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    #1. Remove any existing relationships
    for r in tom.model.Relationships:
        if r.FromTable.Name == "fact_myevents_1bln" and r.ToTable.Name == "dim_Date":
            tom.model.Relationships.Remove(r)

    #2. Creates correct relationships
    tom.add_relationship(from_table="fact_myevents_1bln"                    , from_column="DateKey"     , to_table="dim_Date"       , to_column="DateKey"       , from_cardinality="Many" , to_cardinality="Many")

## Run query on 1Bln Row

In [ ]:
df=fabric.evaluate_dax(
    dataset=ClonedModelName,
    dax_string="""
        EVALUATE
	        SUMMARIZECOLUMNS(
		        dim_Date[DateKey],
		        "Quantity" , [Sum of Sales (1bln)]
		        )
        """
    )
display(df)

## Run query on 2Bln Row
#### This will fail due to guardrail

In [ ]:
df=fabric.evaluate_dax(
    dataset=ClonedModelName,
    dax_string="""
        EVALUATE
	        SUMMARIZECOLUMNS(
		        dim_Date[DateKey],
		        "Quantity" , [Sum of Sales (2bln)]
		        )
        """

    )
display(df)

## Convert cloned model to Direct Lake over SQL Endpoint

In [ ]:
df=pd.DataFrame(labs.list_lakehouses())
endpointid = df[df['Lakehouse Name']==LakehouseName]['SQL Endpoint ID'].iloc[0]
server = df[df['Lakehouse Name']==LakehouseName]['SQL Endpoint Connection String'].iloc[0]

with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:

    #Convert import tables to Direct Lake
    for t in tom.model.Tables:
        for p in t.Partitions:
            if(p.Mode==TOM.ModeType.Import):
                t.Partitions.Remove(p)
                tom.add_entity_partition(table_name=t.Name,entity_name=t.Name)
                print(f"Table {t.Name} converted")
            p.Source.SchemaName=None

    #Switch Model to Direct Lake over SQL
    for e in tom.model.Expressions:
        e.Expression = f"""
        let
            Source = Sql.Database("{server}", "{endpointid}")
        in
            Source"""

print("Converted to Direct Lake over SQL")

## Check what version of Direct Lake is being used

##### Sql.Database    = DirectLake over SQL

##### Azure.Lakehouse = DirectLake over One Lake

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for e in tom.model.Expressions:
        print(e.Expression)

## Show storage mode for each table

In [ ]:
objects = {}
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for t in tom.model.Tables:
        #print(t.Name)
        for p in t.Partitions:
            #print(p.Mode)
            objects[t.Name] = str(p.Mode)
 
df=pd.DataFrame([objects])
display(df)

## Run query on 2bln row table
This should work, but fall back to SQL Endpoint
(Run twice if you get error first time)

In [ ]:
df=fabric.evaluate_dax(
    dataset=ClonedModelName,
    dax_string="""
        EVALUATE
	        SUMMARIZECOLUMNS(
		        dim_Date[DateKey],
		        "Quantity" , [Sum of Sales (2bln)]
		        )
        """
    )
display(df)

## Show TMSL code for cloned model

In [ ]:
import json
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    x= tom.get_bim()

    formatted_json = json.dumps(x, indent=4)
    print(formatted_json)

---

## Lab 8 Summary: Hybrid Storage Mode Mastery

**🎯 What You Accomplished:**
This lab demonstrated the most advanced Direct Lake techniques, showcasing hybrid storage modes that combine Direct Lake over One Lake with Import mode capabilities for maximum flexibility and performance.

**🔧 Key Technical Achievements:**
- **Hybrid Architecture Implementation**: Successfully configured Direct Lake over One Lake with Import mode fallback capabilities
- **Storage Mode Optimization**: Demonstrated conversion between Direct Lake over One Lake and Direct Lake over SQL Endpoint
- **Large-Scale Performance Testing**: Validated query performance on datasets ranging from 1 billion to 2 billion rows
- **Guardrail Management**: Explored Direct Lake guardrails and automatic fallback mechanisms to SQL Endpoint
- **Advanced Relationship Configuration**: Implemented many-to-many relationships for optimal large table performance

**💡 Business Impact:**
- **Scalable Architecture**: Hybrid storage modes provide the best of both worlds - Direct Lake performance with Import mode reliability
- **Cost Optimization**: Intelligent fallback mechanisms ensure queries succeed while maintaining optimal performance characteristics
- **Enterprise Readiness**: Advanced configurations support the largest enterprise datasets with guaranteed query execution
- **Flexibility**: Multiple storage mode options allow for precise performance tuning based on specific business requirements

**🚀 Advanced Concepts Mastered:**
- Direct Lake over One Lake vs. SQL Endpoint implementation differences
- Hybrid storage mode architecture patterns
- Large-scale data performance optimization strategies
- Intelligent query fallback mechanisms

**Next Steps**: You now possess comprehensive knowledge of all Direct Lake storage modes and hybrid architectures, enabling you to design and implement enterprise-scale semantic models that deliver optimal performance across any dataset size while maintaining reliability and cost effectiveness.